# Economic Indicators

This code file is meant to capture the relevant economic indicators for plotting and analysis.

Below are the indicators that have been chosen for analysis:  
* *Empire State Manufacturing Survey*  
* *Philadelphia Business Outlook Survey*  
* *Kansas City Manufacturing Survey of the 10th District*  
* *Richmond Fifth District Survey of Manufacturing Activity*  
* *Chicago Fed National Activity Index*  
* *Yield Curve Spreads*  

Outstanding tasks include:
* Consumer Sentiment data
* Consumer Confidence data
* CPI
* Personal Income & Spending
* Employment numbers
* Z-score calculations
* Plot all Z-scores onto a single graph and compare to stock market
* Clean up the code
* Perform time series lag correlation analysis

In [15]:
# !pip install openpyxl

In [56]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import datetime
from openpyxl import Workbook
import hvplot.pandas
import numpy as np

In [17]:
# Read data
# New York Empire State data
empire_df = pd.read_csv('./Inputs/ESMS_SeasonallyAdjusted_Diffusion.csv')
empire_clean_df = empire_df[['surveyDate','GACDISA']]
empire_clean_df = empire_clean_df.rename(columns={
    "surveyDate":"Date",
    "GACDISA":"Empire_St"
})
empire_clean_df = empire_clean_df.set_index(['Date'])
empire_clean_df.index = pd.to_datetime(empire_clean_df.index)
empire_clean_df.tail()

,Empire_St
Date,
2021-04-30,26.3
2021-05-31,24.3
2021-06-30,17.4
2021-07-31,43.0
2021-08-31,18.3


In [18]:
# Read data
# Philadelphia Business Outlook Survey data
philly_df = pd.read_csv('./Inputs/bos_dif.csv')
philly_clean_df = philly_df[['DATE','GAC']]
philly_clean_df = philly_clean_df.rename(columns={
    "DATE":"Date",
    "GAC":"Philly"
})
# philly_clean_df[['Month','Year']] = philly_clean_df['Date'].str.split('-',expand=True)
# philly_clean_df['Year']='19'+philly_clean_df['Year'].astype(str)
# philly_clean_df['Year'] = philly_clean_df['Year'].apply(lambda x: '19'+philly_clean_df['Year'][x].astype(str) if x > 50,else '20'+philly_clean_df['Year'][x].astype(str))
# '19'+philly_clean_df['Year'].astype(str)
# philly_clean_df.loc[philly_clean_df['Year'] >= 50, 'Year']= '19' + philly_clean_df['Year']
# philly_clean_df.loc[philly_clean_df['Year'] < 50, 'Year']= '20' + philly_clean_df['Year']
philly_clean_df

,Date,Philly
0,May-68,32.2
1,Jun-68,18.4
2,Jul-68,33.3
3,Aug-68,-3.9
4,Sep-68,9.5
...,...,...
635,Apr-21,50.2
636,May-21,31.5
637,Jun-21,30.7
638,Jul-21,21.9


In [19]:
philly_clean_df = philly_clean_df.set_index(['Date'])
philly_clean_df

,Philly
Date,
May-68,32.2
Jun-68,18.4
Jul-68,33.3
Aug-68,-3.9
Sep-68,9.5
...,...
Apr-21,50.2
May-21,31.5
Jun-21,30.7


In [20]:
philly_clean_df.index = pd.to_datetime(philly_clean_df.index, infer_datetime_format=True,format="%b-%y")
philly_clean_df

,Philly
Date,
2068-05-01,32.2
2068-06-01,18.4
2068-07-01,33.3
2068-08-01,-3.9
2068-09-01,9.5
...,...
2021-04-01,50.2
2021-05-01,31.5
2021-06-01,30.7


In [21]:
# Read data
# Kansas City Manufacturing Survey data
kansas_df = pd.read_excel('./Inputs/ManufSurvey_Augu_PR_2021-8-26-Table2.xlsx', engine='openpyxl')
kc_clean_df = kansas_df.drop([0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,
                17,18,19,20,21,22,23,24,25,26,27,28,29,
                30,31,32,33,34,35,37,38,39,40,41,42,43,
                44,45,46,47,48,49,50,51,52,53,54,55,56,
                57,58,59,60,61,62,63,64,65,66,67,68,69,
                70,71,72,73,74,75,76,77,78,79,80,81])
kc_clean_df2 = kc_clean_df.T
kc_clean_df2 = kc_clean_df2.rename(columns={
    1:"Date",
    36:"Composite_Index"})
kc_clean_df2 = kc_clean_df2.drop(['Table2'])
kc_clean_df2 = kc_clean_df2.set_index(['Date'])
kc_clean_df2

,Composite_Index
Date,
2001-07-31,-23
2001-08-31,-12
2001-09-30,-16
2001-10-31,-18
2001-11-30,-24
...,...
2021-04-30,35
2021-05-31,43
2021-06-30,43


In [22]:
# Read data
# Richmond Manufacturing Survey data
rich_df = pd.read_excel('./Inputs/mfg_historicaldata.xlsx', engine='openpyxl')
cols = [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,
        26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,
        46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69]
rich_df.drop(rich_df.columns[cols],axis=1,inplace=True)
rich_clean_df = rich_df.rename(columns={'date':"Date",'sa_mfg_composite':"SA_Mfg_Composite"})
rich_clean_df = rich_clean_df.set_index(['Date'])
rich_clean_df

,SA_Mfg_Composite
Date,
1993-11-01,7
1993-12-01,12
1994-01-01,16
1994-02-01,5
1994-03-01,19
...,...
2021-04-01,22
2021-05-01,24
2021-06-01,26


In [23]:
# Read data
# Chicago Fed National Activity Index data
chicago_df = pd.read_excel('./Inputs/cfnai-data-series-xlsx.xlsx', engine='openpyxl')
cols = [1,2,3,4,6,7]
chicago_df.drop(chicago_df.columns[cols],axis=1,inplace=True)
chicago_df = chicago_df.set_index(['Date'])
chicago_df

,CFNAI
Date,
1967-03-31,-0.33
1967-04-30,0.02
1967-05-31,-0.55
1967-06-30,0.04
1967-07-31,-0.32
...,...
2021-03-31,2.36
2021-04-30,-0.12
2021-05-31,0.15


In [80]:
# Read data
# Yield Curve Treasury rates
rates_df = pd.read_csv('./Inputs/FRB_H15.csv')
cols = [1,3,4,6,8,10]
rates_df.drop(rates_df.columns[cols],axis=1,inplace=True)
rates_df.drop([0,1,2,3,4],inplace=True)
rates_df = rates_df.rename(columns={"Series Description":"Date"})
rates_clean_df = rates_df.rename(columns={rates_df.columns[1] : '3mo_yield',
                                         rates_df.columns[2] : '2yr_yield',
                                         rates_df.columns[3] : '5yr_yield',
                                         rates_df.columns[4] : '10yr_yield',
                                         rates_df.columns[5] : '30yr_yield'})
rates_clean_df = rates_clean_df.set_index(['Date'])
rates_clean_df = rates_clean_df[rates_clean_df['10yr_yield'].str.contains("ND")==False]
rates_clean_df = rates_clean_df[rates_clean_df['3mo_yield'].str.contains("ND")==False]
rates_clean_df = rates_clean_df.apply(pd.to_numeric)
rates_clean_df['10-2_spread'] = rates_clean_df['10yr_yield'] - rates_clean_df['2yr_yield']
rates_clean_df['10-3mo_spread'] = rates_clean_df['10yr_yield'] - rates_clean_df['3mo_yield']
rates_clean_df['30-5_spread'] = rates_clean_df['30yr_yield'] - rates_clean_df['5yr_yield']
cols2 = [0,1,2,3,4]
rates_clean_df.drop(rates_clean_df.columns[cols2],axis=1,inplace=True)
rates_clean_df.index = pd.to_datetime(rates_clean_df.index)
# rates_clean_df.to_csv('int_rate_check.csv')
rates_clean_df.head()

,10-2_spread,10-3mo_spread,30-5_spread
Date,,,
1981-09-01,-1.37,-1.60,-1.42
1981-09-02,-1.38,-1.25,-1.39
1981-09-03,-1.42,-1.48,-1.32
1981-09-04,-1.38,-1.13,-1.33
1981-09-08,-1.36,-0.95,-1.24


In [88]:
# Read data
# Consumer Sentiment data
sentiment_df = pd.read_csv('./Inputs/tbmics.csv',parse_dates={'Date':['YYYY','Month']})
# cols = [1,3,4,6,8,10]
# rates_df.drop(rates_df.columns[cols],axis=1,inplace=True)
# rates_df.drop([0,1,2,3,4],inplace=True)
sentiment_df = sentiment_df.rename(columns={"ICS_ALL":"CS_Index"})
# rates_clean_df = rates_df.rename(columns={rates_df.columns[1] : '3mo_yield',
#                                          rates_df.columns[2] : '2yr_yield',
#                                          rates_df.columns[3] : '5yr_yield',
#                                          rates_df.columns[4] : '10yr_yield',
#                                          rates_df.columns[5] : '30yr_yield'})
sentiment_df = sentiment_df.set_index(['Date'])
# rates_clean_df = rates_clean_df[rates_clean_df['10yr_yield'].str.contains("ND")==False]
# rates_clean_df = rates_clean_df[rates_clean_df['3mo_yield'].str.contains("ND")==False]
# rates_clean_df = rates_clean_df.apply(pd.to_numeric)
# rates_clean_df['10-2_spread'] = rates_clean_df['10yr_yield'] - rates_clean_df['2yr_yield']
# rates_clean_df['10-3mo_spread'] = rates_clean_df['10yr_yield'] - rates_clean_df['3mo_yield']
# rates_clean_df['30-5_spread'] = rates_clean_df['30yr_yield'] - rates_clean_df['5yr_yield']
# cols2 = [0,1,2,3,4]
# rates_clean_df.drop(rates_clean_df.columns[cols2],axis=1,inplace=True)
# rates_clean_df.index = pd.to_datetime(rates_clean_df.index)
# rates_clean_df.to_csv('int_rate_check.csv')
sentiment_df.head()

,CS_Index
Date,
1952-11-01,86.2
1953-02-01,90.7
1953-08-01,80.8
1953-11-01,80.7
1954-02-01,82.0


# New York Empire State Manufacturing Survey

In [24]:
empire_plot = empire_clean_df.hvplot.line(title='New York Empire State Mfg Survey',grid=True)
empire_plot

:Curve   [Date]   (Empire_St)

# Philadelphia Business Outlook Survey

In [25]:
philly_plot = philly_clean_df.hvplot.line(grid=True)
philly_plot

:Curve   [Date]   (Philly)

# Kansas City Manufacturing Survey

In [26]:
kc_plot = kc_clean_df2.hvplot.line(title='Kansas City Mfg Survey - Non Seasonally Adjusted YoY Composite Index',
                                   grid=True)
kc_plot

:Curve   [Date]   (Composite_Index)

# Richmond Fifth District Survey of Manufacturing Activity

In [27]:
rich_plot = rich_clean_df.hvplot.line(title='Richmond Survey of Mfg Activity',
                                   grid=True)
rich_plot

:Curve   [Date]   (SA_Mfg_Composite)

# Chicago Fed National Activity Index

In [28]:
chicago_plot = chicago_df.hvplot.line(title='Chicago Fed National Activity Index',
                                   grid=True)
chicago_plot

:Curve   [Date]   (CFNAI)

# Yield Curve Spreads

In [82]:
yield_curve_plot = rates_clean_df.hvplot.line(title='Yield Curve Spreads',
                                   grid=True)
yield_curve_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

# Consumer Sentiment

In [89]:
sentiment_plot = sentiment_df.hvplot.line(title='Consumer Sentiment',
                                   grid=True)
sentiment_plot

:Curve   [Date]   (CS_Index)